# The Statistics Bureau
####by Omar A. Guerrero, University of Oxford
So far we have created a simple labor market where the only unemployment is the natural unemployment rate. Since our model has a detailed specification at the level of each agent and firm, it allow us to analyze several temporal and cross-sectional aspects of our artificial economy. In order to facilitate the analysis, we will develop a new class of agent: the statistics bureau. This agent will be in charge of performing surveys and census in our population. The implementation of the statistics bureau is not only a natural way to think about the data collection of our model, but it also a cleaner way to perform data collection for policy experiments in the future.

## Libraries
We will add a plotting library: matplotlib

In [1]:
from __future__ import division # forces a division of integers to return a float (in order to avoid errors)
%matplotlib inline
import random as rd # tools to genrate random numbers and more
import matplotlib.pyplot as plt # we will use this to visualize the model's output

## Parameters

In [2]:
L = 1000 # number of workers
N = 100 # number of firms
Lambda = 0.1 # the exogenous source of unemployment, also known as the separation rate
w = .8 # the wage generated (we will assume it to be fixed)
max_t = 500 # maximum number of steps to be ran in the simulation

## Agents

### Workers

In [3]:
class Worker():
    
    # gives birth (initializes) to a single worker
    def __init__(self, id, Lambda=Lambda, w=w, employer=None):
        self.id = id # the id of the worker
        self.Lambda = Lambda # the separation rate of the worker
        self.w = w # the wage of the worker
        self.employer = employer # the firm for which the agent works
    
    # checks if the agent works for a firm and returns False if she is unemployed and True otherwise
    def is_employed(self):
        return not self.employer==None
    
    # sets the wage
    def set_wage(self, w):
        self.w = w
    
    # sets the employer
    def set_employer(self, firm):
        self.employer = firm
        
    # specifies what the agent does in each step of the simulation
    def step(self, t):
        rand = rd.random() # draw a random number
        if self.is_employed(): # check employment status
            # the probability of becoming unemployed is the average of the separation rates of the worker and the firm.
            # This is a bit of an arbitrary assumption but can be modified later. The point is to capture the effect
            # that each counterpart has in the process of unemployment. Since we will assume homogenous agents, this
            # specification does not change from simply assuming the general Lambda to be the only source of unemployment.
            if rand < (self.Lambda + self.employer.Lambda)/2:
                market.separation(self) # peform separation in the market
        else:
            firm = market.pick_firm() # pick a firm at random
            wage = w # determine the wage
            market.hiring(self, firm, wage, t) # perform the hiring in the market

### Firms
We add a new method that the firms use to compute their size.

In [4]:
class Firm():
    
    # initializes the firm
    def __init__(self, id, Lambda=Lambda):
        self.id = id
        self.Lambda = Lambda
        self.book = {}
    
    # register a newly hired worker with her wage and the period of hiring
    def hire(self, worker, wage, step):
        self.book[worker] = (wage, step)
    
    # remove the separated worker from the book
    def separate(self, worker):
        self.book.pop(worker)
    
    # compute the size (in number of employees)
    def size(self):
        return len(self.book)
    
    # specifies what the firm does in each step of the simulation. For now it does nothing
    def step(self, t):
        return # do nothing

### Market

In [5]:
class Market():
    
    # register a hiring in both the firm and the worker attributes
    def hiring(self, worker, firm, wage, step):
        worker.set_employer(firm)
        worker.set_wage(w)
        firm.hire(worker, wage, step)
    
    # register a separation in both the firm and the worker attributes
    def separation(self, worker):
        firm = worker.employer
        firm.separate(worker)
        worker.set_employer(None)
        worker.set_wage(0)
    
    # pick a firm at random
    def pick_firm(self):
        return rd.choice(firms)

### Statistics Bureau
The statistics agent holds a dictionary where all the data structures will be stored. We can also implement a few methods to sun surveys and compute statistics about the entire population. At this point we will perform simple statistics and collect aggregate data. However, as we progress we will use the statistics bureau to collect micro-data  and visualize our synthetic panels.

In [6]:
class Stats():
    
    # initializes the statistics bureau
    def __init__(self):
        self.data = {} # a dictionary that holds different data structures
        self.data['u'] = # the latest unemployment rate
        self.data['ts_u'] = [] # list that stores the time series of the unemployment rate
        
    # counts the number of employed
    def count_employed(self):
        return sum([worker.is_employed() for worker in workers])
    
    # counts the number of unemployed
    def count_unemployed(self):
        return sum([not worker.is_employed() for worker in workers])
    
    # computes the unemployment rate
    def unemployment_rate(self):
        return self.count_unemployed()/L
    
    # computes the average wage
    def average_wage(self):
        return sum([worker.w for worker in workers if worker.is_employed()])/self.count_employed()
    
    # computes the average firm size
    def average_firm_size(self):
        return sum([firm.size() for firm in firms])/N
    
    # what the statistics bureau does every period
    def step(self):
        self.data['u'] = self.unemployment_rate()
        self.data['ts_u'].append(self.data['u'])

SyntaxError: invalid syntax (<ipython-input-6-41fc3ae6c56a>, line 6)

## Execution

### Initialization

In [ ]:
workers = [Worker(id) for id in range(L)] # list of workers parameterized acording to the Parameters section
firms = [Firm(id) for id in range(N)] # list of firms parameterized acording to the Parameters section
agents = workers + firms # make a joint list of workers and firms
rd.shuffle(agents) # randomize the order to the agents in the list
market = Market() # our market object
stats = Stats() # our statistics bureau

### Iteration
We add a call to the statistics bureau step() method

In [ ]:
for t in range(max_t):
    [agent.step(t) for agent in agents]
    stats.step() # the stats agent perform its duty

## Testing
Test the statistics bureau by calling some of its methods and printing the output. 

In [ ]:
# compute unemployment rate
print 'unemployment rate: ' + str(stats.unemployment_rate())

# check that employment + unemployment adds up to L
print 'unemployment+employment=L: ' + str(stats.count_employed() + stats.count_unemployed() == L)

# compute average wage of those employed
print 'average wage: ' + str(stats.average_wage())

# compute average firm size
print 'average firm size: ' + str(stats.average_firm_size())

## Plotting
Implement some basic plots using the data collected by the stats agent.

### Unemployment Rate
Plot the evolution of the unemployment rate. Note that this model has a simple mathematical representation. That is, the number of unemployed people $U_t$ in period $t$ is the result of the previous employed who lost their jobs: $U_t=\lambda (L_{t-1} - U_{t-1})$. In the steady state, the average level of unemployment does not change with time, so $U=\lambda (L - U)$. We can normalize this expression by dividing by th labor force $L$ and denoting the normalized variables by lower case: $u=\lambda (1 - u)$. Finally, we can solve for the unemployment rate $u$ and obtain the analytical prediction: $u=\frac{\lambda}{1+\lambda}$. We test this prediction against the model output graphically in this plot.

In [ ]:
plt.plot(stats.data['ts_u'], linewidth=.5) # plot the time series collected by the stats agent
plt.plot([Lambda/(1+Lambda) for i in range(max_t)], 'r', linewidth=3) # plot the analytical prediction
plt.ylim([min(stats.data['ts_u_rate'][1::]), max(stats.data['ts_u'][1::])])
plt.xlabel('time')
plt.ylabel('unemployment rate')
plt.title('evolution of the unemployment rate')

### Firm Size Distribution
A histogram of the firm size distribution.

In [ ]:
plt.hist([firm.size() for firm in firms]) # plot the time series collected by the stats agent
plt.xlabel('number of employees')
plt.ylabel('frequency')
plt.title('firm size distribution')

## Conclusion
By implementing the stats agent, we were able to verify that the model is consistent with the analytical prediction of the stochastic process that governs the dynamics of this simple theory. The plotting library also facilitates the cross-sectional analysis of the firm population. With these tools in place, we can proceed to introduce other sources of unemployment. 